In [1]:
import pandas as pd
import ee
import numpy as np
import geopandas as gpd
from shapely.geometry import MultiPoint
from config import RECTANGLE, DATES
#ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()

# Extraer datos

This product is the concentration of the photosynthetic pigment (chlorophyll-a) in phytoplankton in the sea surface layer.

This is an ongoing dataset with a latency of 3-4 days.

GCOM-C conducts long-term and continuous global observation and data collection to elucidate the mechanism behind fluctuations in radiation budget and carbon cycle needed to make accurate projections regarding future temperature rise. At the same time, cooperating with research institutions having a climate numerical model, it contributes to reduction of errors in temperature rise prediction derived from the climate numerical model and improvement of accuracy of prediction of various environmental changes. SGLI mounted on GCOM-C is the succession sensor of the Global Imager (GLI) mounted on ADEOS-II (MIDORI II) and is the Imaging Radiometer which measures the radiation from near-ultraviolet to thermal infrared region (380 nm-12 um) in 19 channels. Global observation of once for approximately every two days is possible at mid-latitude near Japan by observation width at ground greater than 1,000 km. In addition, SGLI realizes high resolution than the similar global sensor and has a polarized observation function and a multi-angle observation function.

https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3#description

In [2]:

def download_chlorophyll(date_ini, date_end, rectangle=RECTANGLE, scale=500):
    
    rectangle = ee.Geometry.Rectangle(rectangle)
    rectangle_bounds = rectangle.bounds().getInfo()['coordinates'][0]
    # dataset elected: https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3
    chlor_a_median = ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/CHLA/V3')

    chlor_a_median_clipped = chlor_a_median.filterDate(date_ini, date_end).filterBounds(rectangle).median()#.clip(rectangle)


    # Define the scale and maxPixels parameters for reduceRegion()
    scale = scale
    max_pixels = 1e13
    chlor_a_with_latlon = chlor_a_median_clipped.addBands(chlor_a_median_clipped.pixelLonLat())

    # Get the chlorophyll concentration and coordinates as lists
    chlor_a_data = chlor_a_with_latlon.reduceRegion(
        reducer=ee.Reducer.toList(numOptional=1),
        geometry=rectangle,
        scale=scale,
        maxPixels=max_pixels
    )

    # Get the list of latitudes and longitudes
    latitudes = chlor_a_data.get('latitude').getInfo()
    longitudes = chlor_a_data.get('longitude').getInfo()

    # Get the chlorophyll concentration as a list
    chlor_a_list = chlor_a_data.get('CHLA_AVE').getInfo()

    # Check if the lengths of the lists are the same
    assert len(latitudes) == len(longitudes), "Error: The lengths of the lists are not the same."
    assert len(latitudes) == len(chlor_a_list), "Error: The lengths of the lists are not the same."



    df = pd.DataFrame({'chlor_raw': chlor_a_list, 'lat': latitudes, 'lon': longitudes})
    df['chlor'] = np.log10(df.eval('chlor_raw*0.0016'))
    df = df.dropna()
    
    df.to_csv(f'../app/data/chlorophyll_points_{date_ini}_to_{date_end}.csv', index=False)    


    dict_geometries = {}
    for val in df.chlor.unique():
        geometry = []
        for row in df.query(f'chlor=={val}').index:
            geometry += [(df.loc[row, 'lon'], df.loc[row, 'lat'])]

        dict_geometries[val] = gpd.GeoSeries(MultiPoint(geometry)).convex_hull[0]


    gdf = (pd.DataFrame().from_dict(dict_geometries, orient='index')
                   .reset_index()
                   .rename(columns={'index':'chlor', 0:'geometry'}))
    
    gdf['coordinates'] = gdf.geometry.apply(lambda x: [x.exterior.coords])
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    gdf.to_csv(f'../app/data/chlorophyll_polygons_{date_ini}_to_{date_end}.csv', index=False)
    gdf['chlor'] = gdf.eval('(10**chlor_raw)/0.0016')
    gdf[['chlor', 'chlor_raw', 'geometry']].to_file(f'../app/data/chlorophyll_polygons_{date_ini}_to_{date_end}.json', driver='GeoJSON')
    return gdf


def download_temperature(date_ini, date_end, rectangle=RECTANGLE, scale=500):
    
    rectangle = ee.Geometry.Rectangle(rectangle)
    rectangle_bounds = rectangle.bounds().getInfo()['coordinates'][0]
    # dataset elected: https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_CHLA_V3
    chlor_a_median = ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/SST/V3')

    chlor_a_median_clipped = chlor_a_median.filterDate(date_ini, date_end).filterBounds(rectangle).median()#.clip(rectangle)


    # Define the scale and maxPixels parameters for reduceRegion()
    scale = scale
    max_pixels = 1e13
    chlor_a_with_latlon = chlor_a_median_clipped.addBands(chlor_a_median_clipped.pixelLonLat())

    # Get the chlorophyll concentration and coordinates as lists
    chlor_a_data = chlor_a_with_latlon.reduceRegion(
        reducer=ee.Reducer.toList(numOptional=1),
        geometry=rectangle,
        scale=scale,
        maxPixels=max_pixels
    )

    # Get the list of latitudes and longitudes
    latitudes = chlor_a_data.get('latitude').getInfo()
    longitudes = chlor_a_data.get('longitude').getInfo()
    # Get the chlorophyll concentration as a list
    temp_list = chlor_a_data.get('SST_AVE').getInfo()

    # Check if the lengths of the lists are the same
    assert len(latitudes) == len(longitudes), "Error: The lengths of the lists are not the same."
    assert len(latitudes) == len(temp_list), "Error: The lengths of the lists are not the same."


    df = pd.DataFrame({'temp_raw': temp_list, 'lat': latitudes, 'lon': longitudes})
    df['temp'] = df.eval('temp_raw*0.0012 - 10')
    df = df.dropna()

    dict_geometries = {}
    for val in df.temp.unique():
        geometry = []
        for row in df.query(f'temp=={val}').index:
            geometry += [(df.loc[row, 'lon'], df.loc[row, 'lat'])]

        dict_geometries[val] = gpd.GeoSeries(MultiPoint(geometry)).convex_hull[0]


    df.to_csv(f'../app/data/temperature_points_{date_ini}_to_{date_end}.csv', index=False)    
    gdf = (pd.DataFrame().from_dict(dict_geometries, orient='index')
                   .reset_index()
                   .rename(columns={'index':'temp', 0:'geometry'}))
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    gdf['coordinates'] = gdf.geometry.apply(lambda x: [x.exterior.coords])
    
    gdf.to_csv(f'../app/data/temperature_polygons_{date_ini}_to_{date_end}.csv', index=False)
    gdf[['temp', 'geometry']].to_file(f'../app/data/temperature_polygons_{date_ini}_to_{date_end}.json', driver='GeoJSON')
    return gdf

In [3]:
for i in range(len(DATES)-1):
    df_chlor = download_chlorophyll(DATES[i], DATES[i+1])
    df_temp = download_temperature(DATES[i], DATES[i+1])

UndefinedVariableError: name 'chlor_raw' is not defined

---

In [ ]:
df_chlor

In [ ]:
df_temp#.eval('temp*0.0012 - 10')

In [ ]:
df_chlor

In [ ]:
from keplergl import KeplerGl
df['geometry'] = df['geometry'].apply(lambda x: str(x))

# Initialize a new KeplerGl map
map_1 = KeplerGl(height=500)

# Add data to the map
map_1.add_data(data=df, name='my_data')

# Configure the map
map_1.config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': -30,
            'longitude': -70,
            'zoom': 5
        }
    }
}

# Render the map
map_1

In [ ]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

gdf = df.copy()

# Convert to KeplerGl-compatible DataFrame
df = pd.DataFrame(df.drop('geometry', axis=1))
df['geometry'] = gdf['geometry'].apply(lambda x: x.__geo_interface__)

# Initialize a new KeplerGl map
map_1 = KeplerGl(height=500)

# Add data to the map
map_1.add_data(data=df, name='my_data')
map_1.config(config={
    'version': 'v1',
    'config': {
        'visState': {
            'layers': [{
                'type': 'geojson',
                'data': df.to_json(),
                'config': {
                    'color': [255, 0, 0],
                    'strokeWidth': 2,
                }
            }]
        }
    }
})

# Render the map
map_1


In [ ]:
import math
DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
json = pd.read_json(DATA_URL)
df = pd.DataFrame()

# Custom color scale
COLOR_RANGE = [
    [65, 182, 196],
    [127, 205, 187],
    [199, 233, 180],
    [237, 248, 177],
    [255, 255, 204],
    [255, 237, 160],
    [254, 217, 118],
    [254, 178, 76],
    [253, 141, 60],
    [252, 78, 42],
    [227, 26, 28],
    [189, 0, 38],
    [128, 0, 38],
]

BREAKS = [-0.6, -0.45, -0.3, -0.15, 0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2]


def color_scale(val):
    for i, b in enumerate(BREAKS):
        if val < b:
            return COLOR_RANGE[i]
    return COLOR_RANGE[i]


def calculate_elevation(val):
    return math.sqrt(val) * 10


# Parse the geometry out in Pandas
df["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])
df["valuePerSqm"] = json["features"].apply(lambda row: row["properties"]["valuePerSqm"])
df["growth"] = json["features"].apply(lambda row: row["properties"]["growth"])
df["elevation"] = json["features"].apply(lambda row: calculate_elevation(row["properties"]["valuePerSqm"]))
df["fill_color"] = json["features"].apply(lambda row: color_scale(row["properties"]["growth"]))

In [ ]:
df

In [ ]:
df.head()

In [ ]:
import pydeck as pdk

In [ ]:
view_state = pdk.ViewState(
    **{"latitude": 49.254, "longitude": -123.13, "zoom": 11, "maxZoom": 16, "pitch": 45, "bearing": 0}
)


polygon_layer = pdk.Layer(
    "PolygonLayer",
    df,
    id="geojson",
    opacity=0.8,
    stroked=False,
    get_polygon="coordinates",
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="elevation",
    get_fill_color="fill_color",
    get_line_color=[255, 255, 255],
    auto_highlight=True,
    pickable=True,
)

tooltip = {"html": "<b>Value per Square Meter:</b> {valuePerSqm} <br /><b>Growth rate:</b> {growth}"}

r = pdk.Deck(
    polygon_layer,
    initial_view_state=view_state,
    map_style=pdk.map_styles.LIGHT,
    tooltip=tooltip,
)
r.to_html("polygon_layer.html")